In [1]:
from __future__ import division

In [165]:
import os
import random
import math
import datetime
import time

In [3]:
import numpy as np
import pandas as pd

In [4]:
import tensorflow as tf
from keras import backend as K

Using TensorFlow backend.


In [36]:
features = pd.read_csv('data/features.txt', names=range(42), dtype='float32')
targets = pd.read_csv('data/targets.txt', names=['win', 'lose', 'draw'], dtype='float32')

num_hold_out = 20000

shuff = list(features.index)
random.shuffle(shuff)

holdout_features = features.loc[shuff[:num_hold_out]]
holdout_targets = targets.loc[shuff[:num_hold_out]]

features = features.loc[shuff[num_hold_out:]]
targets = targets.loc[shuff[num_hold_out:]]

In [37]:
features.head()

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
2011976,-1.0,1.0,0.0,0.0,0.0,0.0,-1.0,-1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1155971,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0
1325412,-1.0,1.0,1.0,-1.0,-1.0,0.0,-1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,-1.0,1.0,1.0,0.0
818485,-1.0,-1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,...,-1.0,0.0,0.0,0.0,-1.0,-1.0,0.0,0.0,0.0,0.0
295871,1.0,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [38]:
targets.head()

,win,lose,draw
2011976,0.0,1.0,0.0
1155971,0.0,1.0,0.0
1325412,0.0,1.0,0.0
818485,1.0,0.0,0.0
295871,1.0,0.0,0.0


In [139]:
len(features)

2290475

In [143]:
def get_batch_iter(batch_size, batch_idx, dfs):
    
    length = len(dfs[0])
    for df in dfs:
        assert len(df) == length
        
    batches_per_df = int(math.ceil(length / batch_size))
    
    local_idx = batch_idx % batches_per_df
    
    start = local_idx*batch_size
    end = (local_idx+1)*batch_size
    
    return [df.iloc[start:end] for df in dfs]


def get_batch_random(batch_size, _, dfs):    
    mask = pd.Series(dfs[0].index).sample(n=batch_size, replace=False)    
    return [df.loc[mask] for df in dfs]


def get_batch(batch_size, batch_idx, dfs, how='iter'):
    if how == 'iter':
        return get_batch_iter(batch_size, batch_idx, dfs)
    elif how == 'random':
        return get_batch_random(batch_size, batch_idx, dfs)
    else:
        raise Exception()

In [237]:
def run(graph, output_prefix, batch_size, num_batches, batch_how='iter', save_prefix=None):
    
    with tf.Session(graph=graph) as sess:
    
        K.set_session(sess)
        sess.run(init_op)
    
        suffix = datetime.datetime.now().strftime("%Y_%m_%d_%H:%M:%S")
        name = '{}_{}'.format(output_prefix, suffix)
        train_writer = tf.summary.FileWriter(name, sess.graph)
        
        print "Running {}".format(name)
            
        t = time.time()
        delta_t = 0
        
        for i in range(num_batches):
        
            if i % 1000 == 0:
                
                delta_t = time.time() - t
                t = time.time()
                    
                acc = acc_value.eval(feed_dict={board: holdout_features, outcome: holdout_targets}).mean()
                print "Batch {} Hold-Out Accuracy: {} Time taken: {:.1f}s".format(i, acc, delta_t)

                summary = sess.run(all_summaries, feed_dict={board: holdout_features, outcome: holdout_targets})
                train_writer.add_summary(summary, i)            
            
            batch = get_batch(batch_size, i, [features, targets], how=batch_how)        
            train_step.run(feed_dict={board: batch[0], outcome: batch[1]})
        
        print "DONE TRAINING"
        print "FINAL ACCURACY: {}".format(acc)
        train_writer.close()
        
        if save_prefix:
            save_name = "{}_{}".format(save_prefix, suffix)
            print "SAVING TO: {}".format(save_name)
            os.makedirs(save_name)
            tf.train.Saver().save(sess, save_name+'/model')

## Attempt 1: Normal dense fully-conneected neural network

In [121]:
from keras.layers import Dense
from keras.objectives import categorical_crossentropy
from keras.metrics import categorical_accuracy as accuracy

# Create a graph to hold the model.
graph = tf.Graph()

# Create model in the graph.
with graph.as_default():

    # Keras layers can be called on TensorFlow tensors:
    board = tf.placeholder(tf.float32, shape=(None, 42))
    
    # Fully connected layers
    x = Dense(512, activation='tanh')(board)    
    x = Dense(256, activation='tanh')(x) 
    x = Dense(128, activation='tanh')(x)
    
    # output layer with 10 units and a softmax activation
    preds = Dense(3, activation='softmax')(x) 
    
    outcome = tf.placeholder(tf.float32, shape=(None, 3))
    
    with tf.name_scope('loss') as scope:
        loss = tf.reduce_mean(categorical_crossentropy(outcome, preds))
        tf.summary.scalar('loss', loss)    
    
    with tf.name_scope('accuracy') as scope:
        acc_value = accuracy(outcome, preds)
        tf.summary.scalar('accuracy', tf.reduce_mean(acc_value))
    
    with tf.name_scope('training') as scope:
        train_step = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)
    
    # Initialize all variables
    init_op = tf.global_variables_initializer()
    
    all_summaries = tf.summary.merge_all()

In [122]:
run(graph, './train/dense', batch_size=250, num_batches=50000)

Running ./train/dense_2017_05_27_17:49:04
Batch 0 Hold-Out Accuracy: 0.370950013399
Batch 1000 Hold-Out Accuracy: 0.593599975109
Batch 2000 Hold-Out Accuracy: 0.599399983883
Batch 3000 Hold-Out Accuracy: 0.604799985886
Batch 4000 Hold-Out Accuracy: 0.60900002718
Batch 5000 Hold-Out Accuracy: 0.607699990273
Batch 6000 Hold-Out Accuracy: 0.608749985695
Batch 7000 Hold-Out Accuracy: 0.614250004292
Batch 8000 Hold-Out Accuracy: 0.611549973488
Batch 9000 Hold-Out Accuracy: 0.620450019836
Batch 10000 Hold-Out Accuracy: 0.621500015259
Batch 11000 Hold-Out Accuracy: 0.620549976826
Batch 12000 Hold-Out Accuracy: 0.622600018978
Batch 13000 Hold-Out Accuracy: 0.62374997139
Batch 14000 Hold-Out Accuracy: 0.617500007153
Batch 15000 Hold-Out Accuracy: 0.630800008774
Batch 16000 Hold-Out Accuracy: 0.622900009155
Batch 17000 Hold-Out Accuracy: 0.636200010777
Batch 18000 Hold-Out Accuracy: 0.63504999876
Batch 19000 Hold-Out Accuracy: 0.633849978447
Batch 20000 Hold-Out Accuracy: 0.630599975586
Batch 21

## Attempt 2: A number of parallel CovNets

In [236]:
from keras.layers import Conv2D, Flatten
from keras.layers.core import Reshape
from keras.layers.merge import Concatenate

# Create a graph to hold the model.
graph = tf.Graph()

# Create model in the graph.
with graph.as_default():
    
    # Keras layers can be called on TensorFlow tensors:
    board = tf.placeholder(tf.float32, shape=(None, 42), name='board') 
    outcome = tf.placeholder(tf.float32, shape=(None, 3), name='outcome')
    
    with tf.name_scope('d1') as scope:
        d1 = Dense(512, activation='tanh')(board) 
        
    # The input data is [col0=[row_0, row_1, ...], col1=[row_0, row_1], ...]
    with tf.name_scope('reshape') as scope:
        rs = Reshape((7, 6, 1), input_shape=(42,))(board)    
    
    with tf.name_scope('c1') as scope:
        c1 = Flatten()(Conv2D(1, kernel_size=(4, 4), activation='relu', input_shape=(7, 6, 1))(rs))
    
    with tf.name_scope('c2') as scope:
        c2 = Flatten()(Conv2D(1, kernel_size=(1, 4), activation='relu', input_shape=(7, 6, 1))(rs))

    with tf.name_scope('c3') as scope:
        c3 = Flatten()(Conv2D(1, kernel_size=(4, 1), activation='relu', input_shape=(7, 6, 1))(rs))

    with tf.name_scope('c4') as scope:
        c4 = Flatten()(Conv2D(1, kernel_size=(5, 5), activation='relu', input_shape=(7, 6, 1))(rs))

    with tf.name_scope('c5') as scope:
        c5 = Flatten()(Conv2D(1, kernel_size=(3, 3), activation='relu', input_shape=(7, 6, 1))(rs))
        
    with tf.name_scope('c6') as scope:
        c6 = Flatten()(Conv2D(1, kernel_size=(2, 2), activation='relu', input_shape=(7, 6, 1))(rs))

    with tf.name_scope('merged') as scope:
        merged = Concatenate()([d1, c1, c2, c3, c4, c5, c6])
    
    with tf.name_scope('d2') as scope:
        x = Dense(256, activation='tanh')(merged) 
        
    with tf.name_scope('d3') as scope:
        x = Dense(128, activation='tanh')(x)
    
    # fully-connected layer with 128 units and ReLU activation
    with tf.name_scope('d4') as scope:
        x = Dense(12, activation='tanh')(x)
    
    # output layer with 10 units and a softmax activation
    with tf.name_scope('predictions') as scope:
        preds = Dense(3, activation='softmax', name='preds')(x) 
    
    with tf.name_scope('loss') as scope:
        loss = tf.reduce_mean(categorical_crossentropy(outcome, preds))
        tf.summary.scalar('loss', loss)
        
    with tf.name_scope('accuracy') as scope:
        acc_value = accuracy(outcome, preds)
        tf.summary.scalar('accuracy', tf.reduce_mean(acc_value))
    
    with tf.name_scope('training') as scope:
        train_step = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)    
    
    # Initialize all variables
    init_op = tf.global_variables_initializer()
    
    all_summaries = tf.summary.merge_all()

In [238]:
sess = run(graph, './train/conv2d', batch_size=250, num_batches=5000, save_prefix='./models/cov2d')

Running ./train/conv2d_2017_05_29_14:37:42
Batch 0 Hold-Out Accuracy: 0.307200014591 Time taken: 0.0s
Batch 1000 Hold-Out Accuracy: 0.5950999856 Time taken: 12.4s
Batch 2000 Hold-Out Accuracy: 0.595600008965 Time taken: 12.8s
Batch 3000 Hold-Out Accuracy: 0.601949989796 Time taken: 12.2s
Batch 4000 Hold-Out Accuracy: 0.607949972153 Time taken: 15.7s
DONE TRAINING
FINAL ACCURACY: 0.607949972153
SAVING TO: ./models/cov2d_2017_05_29_14:37:42
[[  4.10964549e-01   5.89025736e-01   9.66553944e-06]]


In [226]:
holdout_features.iloc[0,:].values.reshape(1, 42).shape

(1, 42)

INFO:tensorflow:Restoring parameters from /Users/george/Stats/ConnectFour/models/cov2d_2017_05_29_14:37:42/model
[[  4.10964549e-01   5.89025736e-01   9.66553944e-06]]
